特徴量間の相関を調べる。

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

In [16]:
###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')

In [17]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train.corr()

,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,所在地,部屋数,L,D,K,S
築年数,1.000000,-0.024997,-0.021954,0.035393,-0.161228,-0.212242,-0.020036,0.085138,0.017103,0.224194,-0.169333,0.015629,-0.089629,-0.025648
面積,-0.024997,1.000000,0.198595,0.023916,0.241597,0.305203,0.002166,0.074915,0.005578,0.718179,0.683412,0.687173,0.317683,0.236148
契約期間,-0.021954,0.198595,1.000000,0.402426,0.061712,0.105165,-0.017169,-0.018258,-0.041561,0.138874,0.121148,0.105525,0.036152,0.079338
定期借家,0.035393,0.023916,0.402426,1.000000,0.026725,0.036783,-0.013839,0.069939,0.002089,0.020374,0.000877,0.003912,-0.000884,0.012330
所在階,-0.161228,0.241597,0.061712,0.026725,1.000000,0.776163,0.002179,-0.195676,-0.031526,0.062794,0.221439,0.151762,0.083731,0.077216
全体の階数,-0.212242,0.305203,0.105165,0.036783,0.776163,1.000000,-0.007904,-0.222999,-0.051523,0.069059,0.260433,0.164362,0.094298,0.105392
最寄り駅,-0.020036,0.002166,-0.017169,-0.013839,0.002179,-0.007904,1.000000,-0.164691,0.325510,-0.035293,0.005399,-0.013272,-0.009554,-0.018066
所要時間,0.085138,0.074915,-0.018258,0.069939,-0.195676,-0.222999,-0.164691,1.000000,0.049309,0.162881,0.035268,0.084957,0.055522,0.021091
所在地,0.017103,0.005578,-0.041561,0.002089,-0.031526,-0.051523,0.325510,0.049309,1.000000,0.008998,-0.004970,0.004702,0.010888,-0.013888
部屋数,0.224194,0.718179,0.138874,0.020374,0.062794,0.069059,-0.035293,0.162881,0.008998,1.000000,0.333122,0.473661,0.221163,0.080320


ターゲットも加えて相関行列をみる。

In [18]:
yX = pd.concat([y_train,X_train],axis=1)
yX.corr()

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,所在地,部屋数,L,D,K,S
賃料,1.000000,-0.250612,0.749910,0.190267,0.013303,0.428685,0.494215,0.059439,-0.136089,-0.001000,0.373906,0.563746,0.472382,0.211328,0.195714
築年数,-0.250612,1.000000,-0.024997,-0.021954,0.035393,-0.161228,-0.212242,-0.020036,0.085138,0.017103,0.224194,-0.169333,0.015629,-0.089629,-0.025648
面積,0.749910,-0.024997,1.000000,0.198595,0.023916,0.241597,0.305203,0.002166,0.074915,0.005578,0.718179,0.683412,0.687173,0.317683,0.236148
契約期間,0.190267,-0.021954,0.198595,1.000000,0.402426,0.061712,0.105165,-0.017169,-0.018258,-0.041561,0.138874,0.121148,0.105525,0.036152,0.079338
定期借家,0.013303,0.035393,0.023916,0.402426,1.000000,0.026725,0.036783,-0.013839,0.069939,0.002089,0.020374,0.000877,0.003912,-0.000884,0.012330
所在階,0.428685,-0.161228,0.241597,0.061712,0.026725,1.000000,0.776163,0.002179,-0.195676,-0.031526,0.062794,0.221439,0.151762,0.083731,0.077216
全体の階数,0.494215,-0.212242,0.305203,0.105165,0.036783,0.776163,1.000000,-0.007904,-0.222999,-0.051523,0.069059,0.260433,0.164362,0.094298,0.105392
最寄り駅,0.059439,-0.020036,0.002166,-0.017169,-0.013839,0.002179,-0.007904,1.000000,-0.164691,0.325510,-0.035293,0.005399,-0.013272,-0.009554,-0.018066
所要時間,-0.136089,0.085138,0.074915,-0.018258,0.069939,-0.195676,-0.222999,-0.164691,1.000000,0.049309,0.162881,0.035268,0.084957,0.055522,0.021091
所在地,-0.001000,0.017103,0.005578,-0.041561,0.002089,-0.031526,-0.051523,0.325510,0.049309,1.000000,0.008998,-0.004970,0.004702,0.010888,-0.013888


In [26]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','K','S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a futur

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2341
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 14
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35409.7	valid_1's rmse: 41017.7
[20]	training's rmse: 20985.8	valid_1's rmse: 29980.8
[30]	training's rmse: 15796.7	valid_1's rmse: 26085.4
[40]	training's rmse: 13390.6	valid_1's rmse: 24508
[50]	training's rmse: 12056	valid_1's rmse: 23796.5
[60]	training's rmse: 11139.5	valid_1's rmse: 23322.4
[70]	training's rmse: 10400.3	valid_1's rmse: 22892
[80]	training's rmse: 9664.53	valid_1's rmse: 22571.6
[90]	training's rmse: 9051.47	valid_1's rmse: 22353.7
[100]	training's rmse: 8593.83	valid_1's rmse

,importance
築年数,4.008499e+13
面積,2.431580e+14
契約期間,6.582397e+12
定期借家,1.857613e+12
所在階,4.746837e+12
全体の階数,3.579679e+13
最寄り駅,7.055608e+13
所要時間,8.187477e+12
所在地,2.802448e+13
部屋数,2.472790e+12
